# 主題模型 Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

*<font color = blue>另一種流行的文本分析技術稱為主題建模。 主題建模的最終目標是找到語料庫中存在的各種主題。 語料庫中的每個文件將至少由一個主題（如果不是多個主題）組成。</font>*

*<font color = blue>在本筆記本中，我們將介紹如何進行潛在狄利克雷分配（LDA）的步驟，這是眾多主題建模技術之一。 它是專門為文字資料設計的。</font>*

*<font color = blue>若要使用主題建模技術，您需要提供 (1) 文件術語矩陣和 (2) 您希望演算法選取的主題數量。</font>*

*<font color = blue>應用主題建模技術後，您作為人類的工作是解釋結果並查看每個主題中的單字組合是否有意義。 如果它們沒有意義，您可以嘗試更改主題數量、文件術語矩陣中的術語、模型參數，甚至嘗試不同的模型。</font>*

## Topic Modeling - Attempt #1 (All Text)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/2024.01.05/nlp-in-python-tutorial-master/pickle/dtm_stop.pkl')
data

Mounted at /content/drive


,aaaaah,aaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,...,ze,zealand,zeppelin,zero,zillion,zombie,zombies,zoning,zoo,éclair
ali,0,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,2,...,0,10,0,0,0,0,0,0,0,0
bill,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,1,1,1,1,0,0
dave,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,3,0,0,0,0,1,0,0,...,0,0,0,2,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,0,0
ricky,0,0,0,0,0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,1,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,dave,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0
aaah,0,0,0,1,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,3,0,0
abc,1,0,0,0,0,0,0,0,0,0
abcs,0,0,1,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [6]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
# 我們將把術語文檔矩陣放入新的 gensim 格式，從 df --> 稀疏矩陣 --> gensim 語料庫
cv = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/2024.01.05/nlp-in-python-tutorial-master/pickle/cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

*<font color = blue>現在我們有了語料庫（term-document matrix）和 id2word（dictionary of location: term），我們需要指定另外兩個參數 - 主題數量和傳遞次數。 讓我們從 2 開始主題數量，看看結果是否有意義，然後從那裡增加主題數量。</font>*

In [7]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
# 現在我們有了語料庫（term-document matrix）和 id2word（dictionary of location: term），
# 我們還需要指定另外兩個參數 - 主題數量和傳遞次數
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.040*"and" + 0.035*"the" + 0.025*"you" + 0.021*"to" + 0.016*"that" + 0.015*"in" + 0.015*"of" + 0.014*"it" + 0.012*"was" + 0.010*"my"'),
 (1,
  '0.036*"the" + 0.029*"you" + 0.025*"and" + 0.023*"to" + 0.017*"that" + 0.017*"of" + 0.014*"in" + 0.014*"it" + 0.011*"was" + 0.010*"my"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

*<font color = blue>這些主題看起來不太好。 我們嘗試修改參數。 我們也嘗試修改我們的術語清單。</font>*

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

*<font color = blue>一種流行的技巧是只查看來自一種詞性的術語（僅名詞、僅形容詞等）。 查看賓州大學標籤集：</font>*

In [8]:
# Let's create a function to pull out nouns from a string of text
# 讓我們創建一個函數來從文字字串中提取名詞
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [9]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/2024.01.05/nlp-in-python-tutorial-master/pickle/data_clean.pkl')
data_clean

,transcript
ali,ladies and gentlemen please welcome to the sta...
anthony,thank you thank you thank you san francisco th...
bill,all right thank you thank you very much thank...
dave,this is dave he tells dirty jokes for a living...
jim,ladies and gentlemen please welcome to the ...
joe,ladies and gentlemen welcome joe rogan wha...
john,armed with boyish charm and a sharp wit the fo...
louis,intro fade the music out lets roll hold there ...
mike,wow hey thank you thanks thank you guys hey se...
ricky,hello hello how you doing great thank you wow ...


In [10]:
# 沒有'averaged_perceptron_tagger' 因此要下載
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [11]:
# Apply the nouns function to the transcripts to filter only on nouns
# 將名詞函數應用於轉錄本以僅過濾名詞
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen stage ali hi thank hello na s...
anthony,thank thank people i em i francisco city world...
bill,thank thank pleasure georgia area oasis i june...
dave,jokes living stare work profound train thought...
jim,ladies gentlemen stage mr jim jefferies thank ...
joe,ladies gentlemen joe fck thanks phone fckface ...
john,charm wit snl writer john mulaney marriage bee...
louis,intro music lets lights lights thank i i place...
mike,wow hey thanks look insane years everyone i id...
ricky,hello thank fuck thank im gon youre weve money...


In [12]:
# Create a new document-term matrix using only nouns
# 僅使用名詞建立新的文檔術語矩陣
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aah,abc,abcs,ability,abortion,abortions,abuse,accent,accents,access,...,yulin,yummy,ze,zealand,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,2,0,0,1,0,0,...,0,0,0,10,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,0,0
dave,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,1
louis,3,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
ricky,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0


In [13]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [15]:
# Let's create a function to pull out nouns from a string of text
# 讓我們創建一個函數來從文字字串中提取名詞
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [16]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
dave,dirty jokes living stare most hard work profou...
jim,ladies gentlemen welcome stage mr jim jefferie...
joe,ladies gentlemen joe fck san francisco thanks ...
john,boyish charm sharp wit former snl writer john ...
louis,intro music lets lights lights thank much i i ...
mike,wow hey thanks hey seattle nice look crazy ins...
ricky,hello great thank fuck thank lovely welcome im...


In [17]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
# 僅使用名詞和形容詞建立新的文件術語矩陣，並使用 max_df 刪除常見單字
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,abortions,...,yummy,yyou,ze,zealand,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,0,1,0,0,0,2,0,0,0,...,0,1,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,2,0,...,0,0,0,10,0,0,0,0,0,0
bill,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,1,1,0,0
dave,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
ricky,0,0,0,0,1,1,2,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [18]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [19]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"joke" + 0.004*"fuck" + 0.004*"kid" + 0.004*"mom" + 0.003*"course" + 0.003*"fucking" + 0.003*"clinton" + 0.003*"wife" + 0.003*"parents" + 0.003*"jokes"'),
 (1,
  '0.006*"dude" + 0.005*"fuck" + 0.005*"fucking" + 0.005*"fck" + 0.005*"kid" + 0.004*"everybody" + 0.004*"girl" + 0.004*"ass" + 0.003*"uh" + 0.003*"men"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

*<font color = blue>在我們查看的 9 個主題模型（名詞和形容詞）中，有 4 個主題模型最有意義。 因此，讓我們將其放在這裡​​並通過更多迭代來運行它以獲得更多微調的主題。</font>*

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

These four topics look pretty decent. Let's settle on these for now.

*<font color = blue>這四個題目看起來都還不錯。 讓我們暫時解決這些問題。</font>*

* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [20]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(1, 'ali'),
 (0, 'anthony'),
 (1, 'bill'),
 (0, 'dave'),
 (1, 'jim'),
 (1, 'joe'),
 (0, 'john'),
 (0, 'louis'),
 (1, 'mike'),
 (0, 'ricky')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.